In [ ]:
import pandas as pd
import numpy as np
import difflib # Comparing 
from sklearn.feature_extraction.text import TfidfVectorizer # Convert a collection of raw documents to a matrix of TF-IDF features.
from sklearn.metrics.pairwise import cosine_similarity # Get similarity

In [ ]:
df = pd.read_csv('movies.csv')

In [ ]:
df.head()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


In [ ]:
df.columns

Index(['index', 'budget', 'genres', 'homepage', 'id', 'keywords',
       'original_language', 'original_title', 'overview', 'popularity',
       'production_companies', 'production_countries', 'release_date',
       'revenue', 'runtime', 'spoken_languages', 'status', 'tagline', 'title',
       'vote_average', 'vote_count', 'cast', 'crew', 'director'],
      dtype='object')

In [ ]:
df.shape

(4905, 24)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4905 entries, 0 to 4904
Data columns (total 24 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   index                 4905 non-null   int64  
 1   budget                4905 non-null   int64  
 2   genres                4877 non-null   object 
 3   homepage              1782 non-null   object 
 4   id                    4905 non-null   int64  
 5   keywords              4493 non-null   object 
 6   original_language     4905 non-null   object 
 7   original_title        4905 non-null   object 
 8   overview              4902 non-null   object 
 9   popularity            4905 non-null   float64
 10  production_companies  4905 non-null   object 
 11  production_countries  4905 non-null   object 
 12  release_date          4904 non-null   object 
 13  revenue               4905 non-null   int64  
 14  runtime               4903 non-null   float64
 15  spoken_languages     

In [ ]:
df.isnull().sum()

index                      0
budget                     0
genres                    28
homepage                3123
id                         0
keywords                 412
original_language          0
original_title             0
overview                   3
popularity                 0
production_companies       0
production_countries       0
release_date               1
revenue                    0
runtime                    2
spoken_languages           0
status                     0
tagline                  846
title                      0
vote_average               0
vote_count                 0
cast                      43
crew                       0
director                  30
dtype: int64

Selecting the relevant feature ( Feature Selection )

Content based & Popularity based

In [ ]:
selected_features = ['genres', 'keywords', 'tagline', 'cast', 'director']

In [ ]:
print(selected_features)

['genres', 'keywords', 'tagline', 'cast', 'director']


In [ ]:
# Replacing null values with null strings
for feature in selected_features:
  df[feature] = df[feature].fillna('')

In [ ]:
# Combine the selected features
combined_features = df['genres']+''+df['keywords']+''+df['tagline']+''+df['cast']+''+df['director']

In [ ]:
print(combined_features)

0       Action Adventure Fantasy Science Fictioncultur...
1       Adventure Fantasy Actionocean drug abuse exoti...
2       Action Adventure Crimespy based on novel secre...
3       Action Crime Drama Thrillerdc comics crime fig...
4       Action Adventure Science Fictionbased on novel...
                              ...                        
4900    Action Crime Thrillerunited states\u2013mexico...
4901    Comedy RomanceA newlywed couple's honeymoon is...
4902    Comedy Drama Romance TV Moviedate love at firs...
4903    A New Yorker in ShanghaiDaniel Henney Eliza Co...
4904    Documentaryobsession camcorder crush dream gir...
Length: 4905, dtype: object


In [ ]:
# Convert text data into feature vectors
vectorizer = TfidfVectorizer()

In [ ]:
feature_vectors = vectorizer.fit_transform(combined_features)

In [ ]:
print(feature_vectors)

  (0, 3583)	0.16969494749717556
  (0, 20730)	0.27477717081272773
  (0, 16615)	0.15704171558999927
  (0, 14551)	0.22455190915889925
  (0, 23268)	0.16209230389619905
  (0, 26548)	0.19723141045333212
  (0, 22465)	0.20292106634149282
  (0, 21488)	0.21636438373173644
  (0, 27541)	0.19855955598386538
  (0, 27183)	0.23151548517791487
  (0, 21522)	0.15633244165747598
  (0, 18559)	0.2621239389055514
  (0, 18165)	0.08695667978871793
  (0, 27162)	0.12498764000722043
  (0, 24198)	0.07499439956840201
  (0, 22779)	0.27477717081272773
  (0, 4676)	0.24618275197630565
  (0, 26335)	0.1302943866783312
  (0, 22917)	0.3332459134513912
  (0, 10194)	0.16566267450783573
  (0, 4456)	0.21886225327073855
  (0, 9305)	0.27477717081272773
  (0, 21837)	0.0989339276549797
  (0, 8936)	0.11679023747610974
  (0, 444)	0.08940397118105212
  :	:
  (4903, 5449)	0.31515014578496575
  (4903, 22171)	0.31515014578496575
  (4903, 27362)	0.31515014578496575
  (4903, 27514)	0.30063777611319503
  (4903, 8121)	0.2613163466232658
  (

In [ ]:
# Find the similarity score
similarity = cosine_similarity(feature_vectors)

In [ ]:
print(similarity)

[[1.         0.06762945 0.01450134 ... 0.         0.         0.        ]
 [0.06762945 1.         0.0274973  ... 0.01246013 0.         0.        ]
 [0.01450134 0.0274973  1.         ... 0.         0.         0.        ]
 ...
 [0.         0.01246013 0.         ... 1.         0.         0.        ]
 [0.         0.         0.         ... 0.         1.         0.        ]
 [0.         0.         0.         ... 0.         0.         1.        ]]


In [ ]:
print(similarity.shape)

(4905, 4905)


In [ ]:
# getting movie from the user
movie = input("Enter A movie name: ")

Enter A movie name: iron man


In [ ]:
# Create a list with all the movie names given in the data
title_list = df['title'].tolist()

In [ ]:
print(title_list)

['Avatar', "Pirates of the Caribbean: At World's End", 'Spectre', 'The Dark Knight Rises', 'John Carter', 'Spider-Man 3', 'Tangled', 'Avengers: Age of Ultron', 'Harry Potter and the Half-Blood Prince', 'Batman v Superman: Dawn of Justice', 'Superman Returns', 'Quantum of Solace', "Pirates of the Caribbean: Dead Man's Chest", 'The Lone Ranger', 'Man of Steel', 'The Chronicles of Narnia: Prince Caspian', 'The Avengers', 'Pirates of the Caribbean: On Stranger Tides', 'Men in Black 3', 'The Hobbit: The Battle of the Five Armies', 'The Amazing Spider-Man', 'Robin Hood', 'The Hobbit: The Desolation of Smaug', 'The Golden Compass', 'King Kong', 'Titanic', 'Captain America: Civil War', 'Battleship', 'Jurassic World', 'Skyfall', 'Spider-Man 2', 'Iron Man 3', 'Alice in Wonderland', 'X-Men: The Last Stand', 'Monsters University', 'Transformers: Revenge of the Fallen', 'Transformers: Age of Extinction', 'Oz: The Great and Powerful', 'The Amazing Spider-Man 2', 'TRON: Legacy', 'Cars 2', 'Green Lant

In [ ]:
# Finding movies matching the given movie
close_match = difflib.get_close_matches(movie, title_list)

In [ ]:
print(close_match)

['Iron Man', 'Iron Man 3', 'Iron Man 2']


In [ ]:
res = close_match[0]
print(res)

Iron Man


In [ ]:
# Finding the index of the movie with the given title
movie_index = df[df['title'] == res]['index'].values[0]

In [ ]:
print(movie_index)

68


In [ ]:
# Getting a list of similar movies
similarity_score = list(enumerate(similarity[movie_index]))

In [ ]:
print(similarity_score)
# Prints movie index & Similarity Score

[(0, 0.014850913754238717), (1, 0.03431244026595762), (2, 0.005860445000079892), (3, 0.006241227259798749), (4, 0.014534628085019183), (5, 0.005832405138417291), (6, 0.051324192769466204), (7, 0.14333989886159276), (8, 0.0), (9, 0.034277342507484806), (10, 0.017167194872411497), (11, 0.005270600925131839), (12, 0.0), (13, 0.005505932585784635), (14, 0.043983931354677076), (15, 0.0), (16, 0.18309059217661097), (17, 0.0055928848715181894), (18, 0.014452977766072258), (19, 0.005009689151681074), (20, 0.06527728295354746), (21, 0.005069573121400272), (22, 0.0), (23, 0.0), (24, 0.0), (25, 0.0), (26, 0.15776768968426577), (27, 0.013461843707588711), (28, 0.05574228440319249), (29, 0.006009286746818395), (30, 0.0676935509642909), (31, 0.21985647009978249), (32, 0.02013553722638066), (33, 0.12178041483201614), (34, 0.0), (35, 0.0315820851989938), (36, 0.028952294209138453), (37, 0.0), (38, 0.09828128612306154), (39, 0.07487765551667418), (40, 0.0), (41, 0.014166828089447114), (42, 0.0), (43, 0

In [ ]:
len(similarity_score)

4905

In [ ]:
# Sorting the movies based on Similarity Score
sorted_similar_movies = sorted(similarity_score, key = lambda x:x[1], reverse = True)
print(sorted_similar_movies)

[(68, 1.0000000000000002), (79, 0.23332351818622943), (31, 0.21985647009978249), (16, 0.18309059217661097), (26, 0.15776768968426577), (4503, 0.15306575597233507), (7, 0.14333989886159276), (33, 0.12178041483201614), (46, 0.11771950121056252), (455, 0.11196933496724948), (613, 0.1108709806431258), (3725, 0.1100013914602532), (305, 0.10965465930842677), (174, 0.10653662835703852), (276, 0.10653662835703852), (1312, 0.10103097448861244), (64, 0.10020376126311736), (2337, 0.0995223613129047), (1056, 0.09889991264827339), (85, 0.09852367313528443), (101, 0.09852127920266615), (203, 0.09852127920266615), (38, 0.09828128612306154), (3268, 0.09260351199112507), (1508, 0.09170301250124216), (984, 0.09127090167106697), (890, 0.08983104937143058), (2288, 0.08693161797001467), (2165, 0.08573732935857994), (409, 0.08452978305766076), (1384, 0.08428789298925286), (3545, 0.08424974936691276), (182, 0.08382845914339597), (284, 0.08382845914339597), (458, 0.08039261159548321), (307, 0.0793205099764630

In [ ]:
# print the name of similar movies based on the index

print('Movies suggested for you : \n')

i = 1

for movie in sorted_similar_movies:
  index = movie[0]
  title_from_index = df[df.index==index]['title'].values[0]
  if (i<30):
    print(i, '.',title_from_index)
    i+=1

Movies suggested for you : 

1 . Iron Man
2 . Iron Man 2
3 . Iron Man 3
4 . The Avengers
5 . Captain America: Civil War
6 . The Helix... Loaded
7 . Avengers: Age of Ultron
8 . X-Men: The Last Stand
9 . X-Men: Days of Future Past
10 . Tropic Thunder
11 . X-Men
12 . Made
13 . X2
14 . The Incredible Hulk
15 . The Incredible Hulk
16 . Gothika
17 . X-Men: Apocalypse
18 . A Scanner Darkly
19 . The Judge
20 . Captain America: The Winter Soldier
21 . X-Men: First Class
22 . X-Men: First Class
23 . The Amazing Spider-Man 2
24 . The Best Man
25 . The Nativity Story
26 . Lucky You
27 . Deadpool
28 . The Kite Runner
29 . Arlington Road


MOVIE RECOMMENDATION SYSTEM

In [ ]:
movie_name = input(' Enter your favourite movie name : ')

list_of_all_titles = df['title'].tolist()

find_close_match = difflib.get_close_matches(movie_name, list_of_all_titles)

close_match = find_close_match[0]

index_of_the_movie = df[df.title == close_match]['index'].values[0]

similarity_score = list(enumerate(similarity[index_of_the_movie]))

sorted_similar_movies = sorted(similarity_score, key = lambda x:x[1], reverse = True) 

print('Movies suggested for you : \n')

i = 1

for movie in sorted_similar_movies:
  index = movie[0]
  title_from_index = df[df.index==index]['title'].values[0]
  if (i<30):
    print(i, '.',title_from_index)
    i+=1

 Enter your favourite movie name : The Avengers
Movies suggested for you : 

1 . The Avengers
2 . Avengers: Age of Ultron
3 . Captain America: The Winter Soldier
4 . Iron Man 2
5 . The Incredible Hulk
6 . The Incredible Hulk
7 . Captain America: Civil War
8 . X-Men
9 . Thor
10 . Thor
11 . X-Men: Apocalypse
12 . X2
13 . Thor: The Dark World
14 . Thor: The Dark World
15 . X-Men: The Last Stand
16 . Deadpool
17 . Ant-Man
18 . Ant-Man
19 . The Amazing Spider-Man 2
20 . Iron Man
21 . X-Men: Days of Future Past
22 . The Helix... Loaded
23 . Man of Steel
24 . X-Men: First Class
25 . X-Men: First Class
26 . Superman II
27 . Iron Man 3
28 . Zodiac
29 . A Scanner Darkly


In [ ]:
def multiplication_table(start, stop):
	for x in start:
		for y in st:
			print(str(x*y), end=" ")
		print()

multiplication_table(1, 3)
# Should print the multiplication table shown above

TypeError: ignored